In [ ]:
dataverse = "LocationDb"
table_name = "Locations_RTree_Constant"

In [ ]:
import requests

dbHost = "http://localhost:19002"
adm_path = "/opt/asterixdb/datasets/adm/sports.adm"

load_command = f"""
USE LocationDb; 
LOAD DATASET Locations_RTree_Constant USING localfs
    (("path"="127.0.0.1:///{adm_path}"),("format"="adm"));
"""
response = requests.post(f"{dbHost}/query/service", data={'statement': load_command})
print(response.text)

In [ ]:
count_command = f"""
USE {dataverse};
SELECT COUNT(*) FROM {table_name};
"""
response = requests.post(f"{dbHost}/query/service", data={'statement': count_command})
print(response.json()["results"])

In [ ]:
csv_path = "./data/csv/sports.csv"
query_count = 100_000

with open(csv_path, 'r') as f:
    f.readline()
    for index, line in enumerate(f):
        if index >= query_count:
            break
        line = line.strip()
        if not line:
            continue
        line_parts = line.split(",")
        point = (line_parts[1], line_parts[2])
        points_to_query.append(point)



In [ ]:
import random
from utils import point, rectangle, create_offset_rectangle
query_benchmark_plan_path = f"commands/benchmark_query_plan_{dataverse}_{table_name}_{query_count}.sql"

with open(query_benchmark_plan_path, 'w') as f:
    for _ in range(query_count):
        x, y = random.uniform(-700, 700), random.uniform(-700, 700)
        offset_factor = 2
        offset_x, offset_y = random.uniform(1, offset_factor), random.uniform(1, offset_factor)
        start_x, start_y, end_x, end_y = (x-offset_x, y-offset_y, x+offset_x, y+offset_y)
        query = f"SELECT * FROM {table_name} WHERE spatial_intersect(location, create_rectangle(create_point({start_x},{start_y}), create_point({end_x},{end_y})));"
        f.write(query + "\n") 

In [ ]:
from utils import Logger

logs_filename_parts = [table_name, query_count]
logger = Logger(logs_filename_parts, True)
with open(query_benchmark_plan_path, 'r') as f:
    lines = f.readlines()
    for line in lines:
        line = line.strip()
        if not line:
            continue
        response = requests.post(f"{dbHost}/query/service", data={'statement': line, 'dataverse': dataverse,})
        json_response = response.json()
        if response.status_code != 200:
            logger.error({
                "status": json_response.get("status"),
                "http.status": response.status_code,
                "metrics": json_response.get("metrics"),
                "error": json_response.get("errors"),
            })
        else:
            logger.log({
                "status": json_response.get("status"),
                "http.status": response.status_code,
                "result_count": len(json_response.get("results", [])) if json_response.get("results") else None,
                "metrics": json_response.get("metrics"),
            })